# HTMDEC DMS API Example

This example demonstrates how to use the new HTMDEC DMS `/form` and `/entry` REST API endpoints to search for BIRDSHOT data which can be converted to a Pandas dataframe for analysis and visualization. 

The `birdshot.py` module implements some example utility functions for querying, analyzing, and visualizing BIRDSHOT data.

### Note:
During the ingest process, form data collected by Contextualize in the Campaign 1 structure was converted to the [Campaign 2 structure](https://docs.google.com/document/d/1FpiXwLQi8QAOuLB0Xr80V14qGfqtyCpT/edit). This means that:
* Sample IDs were converted to the Campaign 2 structure 
* Campaign 1 "child" sample numbers (T*nn*) have been replaced with ordinal letters (a, b, c, ...)


### Using the REST API

The [Girder Client](https://girder.readthedocs.io/en/latest/python-client.html) can be used to query the REST endpoints directly. When running via the DMS, the API URL and user token required for access are available in the current environment.


In [1]:
from girder_client import GirderClient
import os

client = GirderClient(apiUrl=os.environ['GIRDER_API_URL'])
client.authenticate(apiKey=os.environ["GIRDER_API_KEY"])

{'_id': '6424afee4236ff9b0883f24b'}

To query VAM data from iteration 1 (AAA) across all forms, use the `/entry/search` endpoint to query the Sample ID. Data is returned in JSON:

Use the `/form` and `entry` endpoints to query data from a specific form:

In [2]:
form_name = 'tensile-details.json'
form = client.get('form', parameters={'entryFileName': form_name, 'limit': 1000})
form[0]['_id']

tensile_data = client.get('entry', parameters={'formId': form[0]['_id'], 'limit': 1000})
tensile_data[0]["data"]

{'Completed By': 'Daniel & Wenle',
 'End Date': '2022-12-13',
 'File Upload': {'file': '',
  'sampleId': 'AAB06_VAM-A_Tensile_a',
  'targetPath': 'AAB/VAM-A/AAB06/Tensile/a'},
 'Modulus Check': {'Force Rate': 10, 'Max Force': 686, 'Max Stress': 225},
 'Notes': '',
 'Results': {'Elastic Modulus': 252,
  'Elongation': 28,
  'Maximum ∂2σ/∂ε2': -1529,
  'UTS/YS Ratio': '1.6794871794871795',
  'Ultimate Tensile Strength': 917,
  'Yield Strength': 546},
 'Sample Dimensions': {'Area': '3.0488',
  'Gauge Length': 8,
  'Thickness': 1.03,
  'Width': 2.96},
 'Start Date': '2022-11-15',
 'Time Spent': '0:04',
 'sampleId': 'AAB06_VAM-A_Tensile_a',
 'suffix': 'Tensile',
 'targetPath': 'AAB/VAM-A/AAB06/Tensile/a'}

In [3]:
iteration = 'CBA'
raw_data = client.get(
        'entry', parameters={'query': f'^{iteration}.._VAM-.', 'limit': 1000}
)
raw_data[0]["data"]

{'Arc Melting': {'3 Part Sections': {'1': 150, '2': 150, '3': 150},
  'Full Ingot': {'1': 200,
   '2': 200,
   '3': 250,
   '4': 250,
   '5': 250,
   '6': 250,
   '7': 250},
  'Ingot Mass Information': {'Final Ingot Mass': 49.7611,
   'Mass Loss': 0.4961707036219271},
  'Process Overview': {'Completed By': 'TAMU VAM Team',
   'Finish Date': '2025-02-05',
   'Start Date': '2025-01-30',
   'Time Spent': '10:00'},
  'Surface': {'1': 130},
  'VAM Details': {'Argon Pressure': 500,
   'Diffusion': 'Before Each Melt',
   'Ingot Location': 0,
   'Initial Purging Times': 3,
   'Vacuum Before Melt': 3e-05}},
 'Material Preparation': {'Process Overview': {'Completed By': 'TAMU VAM Team',
   'Finish Date': '2025-01-30',
   'Start Date': '2025-01-28',
   'Time Spent': '10:00'},
  'Target Mass': {'Co': 4.1669076549424995,
   'Cr': 3.6764157913901787,
   'Fe': 11.845663801911174,
   'Mn': 3.884424615449883,
   'Ni': 22.824738477637585,
   'Total': 50,
   'V': 3.601849658668684},
  'Weighed Mass': {'C

### Using the `birdshot` Module

The `birdshot.py` module implements a few helper functions to query the REST API and convert data into a single Pandas dataframe for analysis:

In [4]:
import birdshot

The `query()` method takes the iteration identifier as an argument and returns a dataframe of results from multiple characterization methods. The example dataframe is intended to reproduce the information available in the Summary Synthesis Results (for example see [HTMDEC AAB Summary Synthesis Results](https://docs.google.com/spreadsheets/d/15cdImpOComsvUpAIq20_nyff65WVzN_q/)).

In [5]:
df = birdshot.query("CBA", client=client)
df

,Target Composition (%).Co,Target Composition (%).Cr,Target Composition (%).Fe,Target Composition (%).Mn,Target Composition (%).Ni,Target Composition (%).V,Elastic Modulus.b,Elongation.b,Maximum ∂2σ/∂ε2.b,UTS/YS Ratio.b,...,Measured Composition (%).Ni,Measured Composition (%).V,XRD.Phase,XRD.Lattice Parameters,Elastic Modulus.d,Elongation.d,Maximum ∂2σ/∂ε2.d,UTS/YS Ratio.d,Ultimate Tensile Strength.d,Yield Strength.d
CBA01_VAM-A,8,8,24,8,44,8,216.734396,0.0,3735.597684,4.311809,...,43.164000,8.198667,FCC,3.592027,NaN,NaN,NaN,NaN,NaN,NaN
CBA02_VAM-A,16,8,8,12,44,12,215.335167,0.0,-125948.206301,1.654387,...,42.989286,12.310000,FCC,3.596722,206.448554,0.0,-2374.210897,1.817676,639.557427,351.854392
CBA03_VAM-A,32,8,8,12,32,8,240.048353,0.0,-676.891803,3.898302,...,31.064667,8.200667,FCC,3.581590,NaN,NaN,NaN,NaN,NaN,NaN
CBA04_VAM-A,8,8,20,12,44,8,211.327992,0.0,3415.666749,4.709260,...,43.207333,8.176667,FCC,3.592766,188.731143,0.0,2508.527092,4.376715,830.479644,189.749518
CBA05_VAM-A,16,8,16,8,44,8,197.640518,0.0,1284.138229,3.542088,...,43.158667,8.188000,FCC,3.581211,NaN,NaN,NaN,NaN,NaN,NaN
CBA06_VAM-A,8,8,32,8,40,4,207.703503,0.0,-28274.541465,1.611988,...,39.069333,4.089333,FCC,3.590217,216.237784,0.0,-13665.960844,1.660185,680.576583,409.940171
CBA07_VAM-A,16,8,12,12,40,12,232.753835,0.0,-12170.937095,1.642333,...,39.183333,12.272667,FCC,3.594124,216.521794,0.0,764.573415,3.816375,980.292708,256.864871
CBA08_VAM-A,16,8,12,12,40,12,233.989336,0.0,510.672513,4.011819,...,39.105714,12.157857,FCC,3.587536,NaN,NaN,NaN,NaN,NaN,NaN
CBA09_VAM-A,28,12,12,8,36,4,207.192000,0.0,1391.048636,4.577674,...,35.323333,4.083333,FCC,3.573890,NaN,NaN,NaN,NaN,NaN,NaN
CBA10_VAM-A,12,8,20,8,44,8,227.559923,0.0,-579.530839,3.820345,...,43.089333,8.182667,FCC,3.587710,NaN,NaN,NaN,NaN,NaN,NaN
